In [1]:
from pathlib import Path
from urllib.request import urlretrieve
import zipfile
from imageio import imread
from scipy.ndimage.interpolation import zoom#
import numpy as np


def fetch_cropped_yaleb(data_folder, zooming=0.5, max_n_subjects=None):
    """Returns a dictionary of paths
    
    Parameters
    ----------
    data_folder: string
    zooming: float, optional, default is 0.5
        factor by which to resize the images
    max_n_subjects: {None, int}, optional, default is None
        if not None, only the first max_n_subjects are returned
    
    Returns
    -------
    dict: {
        subjects_1: {'images': [image_1, ... image_N],
               'ambient': image_ambient,
        }
    }
    
    images are stored as numpy arrays
    """
    url = 'http://vision.ucsd.edu/extyaleb/CroppedYaleBZip/CroppedYale.zip'
    yaleb_path = Path(data_folder).joinpath('cropped_yaleb')
    
    if not yaleb_path.joinpath('CroppedYale').exists():
        yaleb_path.mkdir(parents=True)
    
    # If not already unzip, do it
    if not list(yaleb_path.iterdir()):
        zip_path = yaleb_path.joinpath('yaleb.zip')
        
        # If zip not already downloaded, download it
        if not zip_path.exists():
            urlretrieve(url, zip_path.as_posix())
        
        zfile = zipfile.ZipFile(zip_path.as_posix())
        zfile.extractall(path=yaleb_path.as_posix())

    yaleb = {}
    for folder_path in yaleb_path.joinpath('CroppedYale').iterdir():
        if max_n_subjects is not None and len(yaleb) > max_n_subjects:
            return yaleb
        
        if not folder_path.is_dir():
            continue
            
        video_name = folder_path.name
        paths = sorted(list(folder_path.glob('*.pgm')))
        images = []
        for path in paths:
            if 'Ambient' in path.name:
                ambient = imread(path.as_posix())
            else:
                images.append(zoom(imread(path.as_posix()), zooming)[None, ...])
                
        data = {'images':np.concatenate(images),
        'ambient':ambient}
        yaleb[video_name] = data

    return yaleb


In [2]:
data = fetch_cropped_yaleb('yaleb/', zooming=0.3, max_n_subjects=5)

/home/scopatz/miniconda/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


In [3]:
X = np.concatenate([data[key]['images'] for key in data], axis=0)
print(X.shape)

(384, 58, 50)


In [4]:
X = X.astype(np.float64)
X -= X.mean()

In [5]:
from tensorly.random import check_random_state
import sparse

random_state = 1234

## Try Sparse

In [6]:
from tensorly.contrib.sparse.decomposition import robust_pca as sparse_robust_pca

In [7]:
Y = sparse.COO(X)
Y

<COO: shape=(384, 58, 50), dtype=float64, nnz=1113600, fill_value=0.0>

In [8]:
low_rank_part, sparse_part = sparse_robust_pca(Y, reg_E=0.04, learning_rate=1.2, n_iter_max=20)

Partial SVD J[0]
Partial SVD J[1]
Partial SVD J[2]
Partial SVD J[0]
Partial SVD J[1]
Partial SVD J[2]


ValueError: Inconsistent fill-values in the result array: operating on the ndarray withfill-values produces inconsistent results.

In [ ]:
print('X.shape={} == low_rank_part.shape={} == sparse_part.shape={}.'.format(Y.shape, low_rank_part.shape, sparse_part.shape))